In [1]:
import math
import string
import itertools
from collections import Counter

## Skytale-Chiffre

$M = M_1 ... M_n$ Klartext
<br>
$C = C_1 ... C_n$ Geheimtext
<br>
$B$ Blockgröße
<br>
$i = \frac{n}{B}$ Anzahl Zeilen

### Verschlüsselung
$A = \begin{bmatrix}
m_{11} & .. & m_{1B}\\
.. &  & ..\\
m_{i1} & .. & m_{iB}
\end{bmatrix}$ Klartext in Matrixdarstellung mit $B$ Spalten und $i$ Zeilen
<br>

$A^T = \begin{bmatrix}
m_{11} & .. & m_{i1}\\
.. &  & ..\\
m_{1B} & .. & m_{iB}
\end{bmatrix}$ Klartext in transponierter Matrixdarstellung, welcher Zeilenweise als Geheimtext ausgegeben wird
<br>

### Entschlüsselung
Die Entschlüsselung verläuft analog zur Verschlüsselung allerdings mit einer neuen Blocklänge $B_c = \frac{n}{B}$ 

In [2]:
def scytale_encryption(clear_text, block_size):
    # add filling characters if needed
    modulo = len(clear_text) % block_size
    if(modulo != 0):
        clear_text = clear_text + "." * (block_size - modulo)
            
    matrix = list(map(''.join, zip(*[iter(clear_text)]*block_size)))
    transposed_matrix = [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]
    
    cipher_text = ''.join(str(character) for row in transposed_matrix for character in row)
    return cipher_text
    
def scytale_decryption(cipher_text, block_size):
    # calculate new column count
    block_size = math.ceil(len(cipher_text) / block_size)
    return scytale_encryption(cipher_text, block_size)

In [3]:
cipher_alphabet = {character: index for index, character in enumerate(string.ascii_lowercase)}
cipher_alphabet_inverse = {index: character for index, character in enumerate(string.ascii_lowercase)}

## Caesar-Chiffre

$M = M_1 ... M_n$ Klartext
<br>
$C = C_1 ... C_n$ Geheimtext
<br>
$K$ Schlüssel

$E_K(M_i)$ Verschlüsselung
<br>
$D_K(C_i)$ Entschlüsselung
<br>
### Verschlüsselung
$C_i = E_K(M_i) = (M_i + K)\ mod\ 26$
### Entschlüsselung
$M_i = D_K(C_i) = (C_i - K)\ mod\ 26$

In [4]:
def caesar_cipher(clear_text, offset, encrypt):
    clear_text = clear_text.lower()
    cipher_text = ""
    
    for index,char in enumerate(clear_text):
        if char not in string.ascii_lowercase:
            cipher_text = cipher_text + char
            continue
        
        clear_index = cipher_alphabet[char]
        cipher_index = None
        if(encrypt):
            cipher_index = (clear_index + offset) % len(cipher_alphabet)
        else:
            cipher_index = (clear_index - offset) % len(cipher_alphabet)
        
        cipher_text = cipher_text + cipher_alphabet_inverse[cipher_index]
        
    return cipher_text


## Vigenère-Chiffre

$M = M_1 ... M_n$ Klartext
<br>
$C = C_1 ... C_n$ Geheimtext
<br>
$K = K_1 ... K_m$ Schlüssel, welcher $\frac{n}{m}$ mal wiederholt wird.

$E_K(M_i)$ Verschlüsselung
<br>
$D_K(C_i)$ Entschlüsselung
<br>
### Verschlüsselung
$C_i = E_K(M_i) = (M_i + K_i)\ mod\ 26$
### Entschlüsselung
$M_i = D_K(C_i) = (C_i - K_i)\ mod\ 26$

In [5]:
def vigenere_cipher(clear_text, cipher, encrypt):
    clear_text = clear_text.lower()
    cipher = cipher.lower()
    cipher_text = ""
    
    # perform a caesar encryption with an offset specified by an index of the cipher character
    for (char, cipher_key) in zip(clear_text,itertools.cycle(cipher)):
        cipher_text = cipher_text + caesar_cipher(char, cipher_alphabet[cipher_key], encrypt)
    return cipher_text

## Buchstabenfrequenzanalyse

$M = M_1 ... M_n$ Klartext
<br>

$n$ Anzahl der Zeichen im Text
<br>

$L$ Buchstabe aus Alphabet
<br>

$n_L$ Anzahl der Zeichen $L$ im Text $M$
<br>

$P(L) = \frac{n}{n_L}$


In [6]:
def letter_frequency_analysis(text):
    text = text.lower()
    text_length = len(text)
    counter = Counter(text)
    
    # fill the 'probabilities' dict with each letter as a key and it's probability in the text
    # as the corresponding value
    return { character: (lambda character :counter[character] / text_length)(character)
            for character in string.ascii_lowercase}  

## Caesar-Kryptoanalyse

$M = M_1 ... M_n$ Klartext
<br>

$C = C_1 ... C_n$ Geheimtext
<br>

$L_C = L$ mit $max(P(L))$ aus $C$ (Zeichen mit größter Wahrscheinlichkeit)
<br>
$K_C =$ Index von $L_C$ aus Alphabet
<br>
$L_R =$ Häufigst vorkommender Buchstabe der Referenzsprache
<br>
$K_R =$ Index von $L_R$ aus Alphabet
<br>
$K = K_C - K_R$ Für Verschlüsselung verwendeter Offset
<br>

$M = E_{K}(C)$ 
<br>


In [7]:
def caesar_cryptoanalysis(cipher_text, reference_letter_probabilities):
    cipher_letter_probabilities = letter_frequency_analysis(cipher_text)
    
    # sort to get the letter from the cipher text with the highest probability 
    cipher_letter_propabilities = dict(sorted(cipher_letter_probabilities.items(), key=lambda item: item[1], reverse=True))
    
    # get the letter from the reference with the highest probability
    alphabet_letter_propabilities = dict(sorted(reference_letter_probabilities.items(), key=lambda item: item[1], reverse=True))
    
    # get the most probable offset 
    sorted_cipher_key_list = list(cipher_letter_propabilities.keys())
    sorted_letter_list = list(alphabet_letter_propabilities.keys())
    offset = cipher_alphabet[sorted_cipher_key_list[0]] - cipher_alphabet[sorted_letter_list[0]]
    
    return(offset)

## Koinzidenzindex

$n_i$ Anzahl der Vorkommnisse des Zeichens $i$ im Text
<br>
$N$ Anzahl der Zeichen im Text 
<br>
$IC = \frac{\sum_{i = A}^{Z} n_i(n_i-1)}{N (N-1)}$

In [8]:
# get the specific index of coincidence of a given text 
def index_of_coincidence(text):
    if len(text) == 1:
        return 0
    
    text = text.lower()
    text_length = len(text)
    counter = Counter(text)
    
    sum = 0
    for letter in string.ascii_lowercase:
        letter_count = counter[letter]
        sum += letter_count * (letter_count -1)
    divisor = (text_length * (text_length-1) )
    return sum / divisor
    

In [9]:
# calculate the average index of coincidence of text segments for a given cipher length
def calculate_index_of_coincidence_per_segment(text, cipher_length):
    text_segments = [''] * cipher_length
    for ((index, char), cipher_index) in zip(enumerate(text),itertools.cycle(range(0, cipher_length))):
        text_segments[cipher_index] += char 
        
    return sum([index_of_coincidence(segment)for segment in text_segments]) / cipher_length

## Vigenere-Kryptoanalyse


$M = M_1 ... M_n$ Klartext
<br>

$C = C_1 ... C_n$ Geheimtext
<br>

### Schlüssellänge finden
Berechne den durchschnittlichen Koinzidenzindex der einzelnen Textsegmente für Schlüssellängen 1 bis 30
und berechne die Differenz zum Koinzidenzindex der verwendeten Sprache.
Über den zur Referenz nahesten Koinzidenzindex wird die gesuchte Schlüssellänge ermittelt.


### Schlüsselzeichen finden
Um jedes Zeichen des Schlüssels zu finden wird die Buchstabenfrequenzanalyse der Caesar-Kryptoanalyse verwendet. 
<br>


In [10]:
def vigenere_cryptoanalysis(cipher_text, reference_coincidence_index, reference_letter_probabilities):
    # find the cipher length by calculating the index of coindicende for various cipher lengths
    indices_of_coincidence = {i : calculate_index_of_coincidence_per_segment(cipher_text,i) for i in range(1,30)}
    
    # select the most probable cipher length
    difference_from_reference = {key: abs(indices_of_coincidence[key] - reference_coincidence_index) for key in indices_of_coincidence.keys()}
    minimum_difference = min(difference_from_reference.values())
    most_probable_length = min(difference_from_reference, key=difference_from_reference.get)
    
    
    # find the key
    key = ''
    # split text into segments
    text_segments = [''] * most_probable_length
    for ((index, char), cipher_index) in zip(enumerate(cipher_text),itertools.cycle(range(0, most_probable_length))):
        text_segments[cipher_index] += char    
    
    for text_segment in text_segments:
        # perform caesar cryptanalysis for key specific segment
        offset = caesar_cryptoanalysis(text_segment, reference_letter_probabilities)
        # translate the offsets to keys and add them to the list of probable keys
        key += cipher_alphabet_inverse[offset]
        
    return key

In [11]:
with open("bielefeld.txt", mode="r", encoding="utf-8") as f:
    bielefeld_text = f.read()

print_length = 100

german_coincidence_index = 0.076
german_text_letter_probabilities = {
        "e": 0.1740,
        "n": 0.0978,
        "i": 0.0755,
        "s": 0.0727,
        "r": 0.0700,
        "a": 0.0651,
        "t": 0.0615,
        "d": 0.0508,
        "h": 0.0476,
        "u": 0.0435,
        "l": 0.0344,
        "c": 0.0306,
        "g": 0.0301,
        "m": 0.0253,
        "o": 0.0251,
        "b": 0.0189,
        "w": 0.0189,
        "f": 0.0166,
        "k": 0.0121,
        "z": 0.0113,
        "p": 0.0079,
        "v": 0.0067,
        "ß": 0.0031,
        "j": 0.0027,
        "y": 0.0004,
        "x": 0.0003,
        "q": 0.0002,
    }


#### Skytale (Test and timing)
##### Verschlüsselung (vorher: 277 ms +- 155 ms)

In [12]:
%timeit scytale_encryption(bielefeld_text, 16)

40.3 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
cipher_text = scytale_encryption(bielefeld_text, 16)
print(cipher_text[:print_length])

BlfdileßulNl3nöeeehNld a bsidnB nc.h drs errue dre reeuutaelHnrcn u rdin eni
eegrn  eaerez heelleu l


##### Entschlüsselung (vorher: 71.8 ms +- 32.6 ms)

In [14]:
%timeit scytale_decryption(cipher_text, 16)

65.3 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
clear_text = scytale_decryption(cipher_text, 16)
print(clear_text[:print_length])

Bielefeld  [ˈbiːləfɛlt] (ostwestfälisch Builefeld, Bielefeld, Beilefeld oder Builefeild) ist eine kr


#### Caesar (Test and Timing)
##### Verschlüsselung (vorher: 248 ms +- 111 ms)

In [16]:
%timeit caesar_cipher(bielefeld_text, 15, True)

88.8 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
cipher_text = caesar_cipher(bielefeld_text, 15, True)
print(cipher_text[:print_length])

qxtatutas  [ˈqxːaəuɛai] (dhilthiuäaxhrw qjxatutas, qxtatutas, qtxatutas dstg qjxatutxas) xhi txct zg


##### Entschlüsselung (vorher: 318 ms +- 143 ms)

In [18]:
%timeit caesar_cipher(cipher_text, 15, False)

96.8 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
clear_text = caesar_cipher(cipher_text, 15, False)
print(clear_text[:print_length])

bielefeld  [ˈbiːləfɛlt] (ostwestfälisch builefeld, bielefeld, beilefeld oder builefeild) ist eine kr


#### Caesar-Kryptoanalyse (Test and Timing)

In [20]:
%timeit caesar_cryptoanalysis(cipher_text, german_text_letter_probabilities)

8.96 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
deciphered_offset = caesar_cryptoanalysis(cipher_text, german_text_letter_probabilities)
print("The most probable offset for the cipher text is: {}".format(deciphered_offset))

The most probable offset for the cipher text is: 15


#### Vigenere
##### Verschlüsselung (vorher: 301 ms +- 107 ms)

In [22]:
vigenere_key = "Testschluessel"
len(vigenere_key)

14

In [23]:
%timeit vigenere_cipher(bielefeld_text, vigenere_key, True)

177 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
cipher_text = vigenere_cipher(bielefeld_text, vigenere_key, True)
print(cipher_text[:print_length])

umwewhlwx  [ˈmbːdəxɛse] (gwepikmxästmgz ffbpwywnk, fawppyidw, ipcpwxiww gwwt momdwjpbpv) kze iafi dv


#### Entschlüsselung (vorher: 326 ms +- 133 ms)

In [25]:
%timeit vigenere_cipher(cipher_text, vigenere_key, False)

173 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
clear_text = vigenere_cipher(cipher_text, vigenere_key, False)
print(clear_text[:print_length])

bielefeld  [ˈbiːləfɛlt] (ostwestfälisch builefeld, bielefeld, beilefeld oder builefeild) ist eine kr


#### Vigenere-Kryptoanalyse (Test and Timing)

In [27]:
%timeit vigenere_cryptoanalysis(cipher_text, german_coincidence_index, german_text_letter_probabilities)

4.11 s ± 302 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
deciphered_key = vigenere_cryptoanalysis(cipher_text, german_coincidence_index, german_text_letter_probabilities)
print("The deciphered key for the cipher text is: " + deciphered_key)

The deciphered key for the cipher text is: testschluessel
